# AI Portfölj 1 – Colab Kontrollpanel

In [ ]:
!pip install pandas yfinance matplotlib openpyxl

In [ ]:
import os, pandas as pd, requests, io
from datetime import date, timedelta

PORTFOLIO_NAME = "AI Portfölj 1"
CURRENCY = "SEK"
INITIAL_CASH = 5000.0
REPO_RAW = "https://raw.githubusercontent.com/<user>/ai-portfolio-1/main"
EOD_FILE = "/content/eod_log.csv"

def fetch_latest_prices():
    for d in [date.today(), date.today()-timedelta(days=1), date.today()-timedelta(days=2)]:
        url = f"{REPO_RAW}/data/daily/prices_raw_{d.strftime('%Y%m%d')}.csv"
        r = requests.get(url)
        if r.ok and len(r.text)>50:
            return pd.read_csv(io.StringIO(r.text)), url
    raise RuntimeError("Ingen dagsfil hittad.")
prices_raw, src = fetch_latest_prices()
print("Hämtade från:", src)
prices_raw.head()


In [ ]:
def as_float(x):
    try: return float(str(x).replace(",", "."))
    except: return float("nan")

holdings = pd.read_csv("/content/holdings.csv")
for col in ["Antal","Inköpskurs"]:
    holdings[col] = holdings[col].apply(as_float)

px = prices_raw.rename(columns={"ticker":"Ticker"})
holdings = holdings.merge(px[["Ticker","close_sek"]], on="Ticker", how="left")
holdings["Marknadsvärde"] = (holdings["Antal"] * holdings["close_sek"]).fillna(0.0)

def stop_hit(row):
    sl = str(row.get("Stop-loss",""))..strip()
    price, buy = row["close_sek"], row["Inköpskurs"]
    if pd.isna(price) or pd.isna(buy): return False
    if sl.endswith("%"):
        try: p = float(sl[:-1]) / 100.0
        except: return False
        return price <= (1 - p) * buy
    try: return price <= float(sl)
    except: return False

holdings["StopLossTriggad"] = holdings.apply(stop_hit, axis=1)
holdings


In [ ]:
def ensure_eod(path):
    cols = ["date","day_tag","portfolio_name",f"cash_{CURRENCY}",f"total_value_{CURRENCY}","notes"]
    if not os.path.exists(path):
        pd.DataFrame(columns=cols).to_csv(path, index=False)

def upsert_eod(path, cash, total, notes=""):
    ensure_eod(path)
    df = pd.read_csv(path)
    today = date.today().isoformat()
    row = {"date": today,"day_tag": f"Dag X – {today}",
           "portfolio_name": PORTFOLIO_NAME,
           f"cash_{CURRENCY}": float(cash),
           f"total_value_{CURRENCY}": float(total),
           "notes": notes}
    if (df["date"] == today).any():
        df.loc[df["date"] == today] = row
    else:
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df.to_csv(path, index=False)
    return df

total_mv = float(holdings["Marknadsvärde"].sum())
cash = INITIAL_CASH
eod = upsert_eod(EOD_FILE, cash, cash+total_mv, notes="EOD uppdaterad.")
eod.tail()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(EOD_FILE, parse_dates=["date"]) 
plt.plot(df["date"], df[f"total_value_{CURRENCY}"])
plt.title(f"EOD Total Value – {PORTFOLIO_NAME}")
plt.grid(True); plt.show()
